In [1]:
import pandas as pd

In [131]:
import numpy as np

In [164]:
import os

In [236]:
import pickle

In [167]:
data_path = "~/Documents/External_Disk_Link_WD_HDD/AHS_Data/"

In [198]:
data_path = "/Volumes/WD HDD/AHS_Data/"

In [199]:
AHS_struct_workbook = pd.ExcelFile(data_path + "Data_structure_AHS.xlsx")

In [3]:
AHS_struct_workbook.sheet_names

[u'File Name',
 u'State District Codes',
 u'Schedule Codes',
 u'COMB',
 u'MORT',
 u'WOMAN',
 u'WPS']

In [4]:
AHS_struct_sheets_names = AHS_struct_workbook.sheet_names

In [5]:
def get_sheet_field_names(excel_workbook, sheet_name) :
    # Start from row 3, as initial 2 rows contain no info
    sheet = excel_workbook.parse(sheet_name, skiprows=2, na_values=['NA'])
    # Find index of 'NOTES:' in 1st cloumn and delete all rows below it
    notes_index = sheet.loc[sheet['Field Order'] == "NOTES:"].index.tolist()[0]
    sheet = sheet.ix[1 : notes_index - 1]
    
    # select column 2,3 and 4 (Filed name, Description and Codes used)
    sheet = sheet[[1,2,3]]
    # Remove <NaN> from Field Names
    sheet = sheet.dropna(subset=[list(sheet)[0]])
    
    # Selecting Non-Yellow field names
    # Dropping <NaN> from Field Descriptions and Codes Used
    sheet_non_yellow = sheet.dropna(subset=[list(sheet)[1], list(sheet)[2]])
    
    # Selecting 'None' and Non-'None' Codes used
    sheet_code_not_none = sheet_non_yellow[sheet_non_yellow['Codes Used'] != "None"]
    sheet_code_none = sheet_non_yellow[sheet_non_yellow['Codes Used'] == "None"]
    
    # Convert all 'Field Names' to list()
    sheet_all = sheet['Field Name'].tolist()
    sheet_non_yellow = sheet_non_yellow['Field Name'].tolist()
    sheet_yellow = list(set(sheet_all) - set(sheet_non_yellow))
    sheet_code_not_none = sheet_code_not_none['Field Name'].tolist()
    sheet_code_none = sheet_code_none['Field Name'].tolist()
    
    # Output in form of list() of lists()
    output = list()
    output.append(sheet_yellow)
    output.append(sheet_non_yellow)
    output.append(sheet_all)
    output.append(sheet_code_none)
    output.append(sheet_code_not_none)
    # output = list[sheet_yellow, sheet_non_yellow, sheet_all, sheet_code_none, sheet_code_not_none]
    
    return(output)

In [7]:
# Turn characters of list into lowercase and limit to max 32 char
def lowercase_32Char(list_):
    list_1 = [x.lower() for x in list_]
    list_2 = [x[0:32] for x in list_1]
    return (list_2)

In [8]:
def lowercase_32Char_list(field_list) :
    # Field names in CSV files are max upto 32 characters
    # and all small letters
    l = len(field_list)
    sol = list()
    
    for field in field_list:
        sol.append(lowercase_32Char(field))
    
    return(sol)

In [69]:
# Remove yellow fields from the data frame
def remove_yellow_fields(data_frame, yellow_field_list) :
    df_col_names = list(data_frame)
    
    drop_col = list()
    for yellow in yellow_field_list :
        if yellow in df_col_names :
            drop_col.append(yellow)
            
    df = data_frame.drop(drop_col, axis=1)
    
    if 'id' in list(df) :
        df = df.drop(['id'], axis=1)
    
    return df

In [88]:
def sort_dataset_state_dist_house(data_frame) :
    return (data_frame.sort(['state', 'district', 'house_no', 'house_hold_no'])).reset_index(drop=True)

In [233]:
mort_field_list = lowercase_32Char_list(get_sheet_field_names(AHS_struct_workbook, "MORT"))
wps_field_list = lowercase_32Char_list(get_sheet_field_names(AHS_struct_workbook, "WPS"))
comb_field_list = lowercase_32Char_list(get_sheet_field_names(AHS_struct_workbook, "COMB"))
women_field_list = lowercase_32Char_list(get_sheet_field_names(AHS_struct_workbook, "WOMAN"))

In [90]:
AHS_mort = pd.read_csv(data_path + "22_AHS_MORT.csv", sep="|")

In [228]:
AHS_wps = pd.read_csv(data_path + "22_AHS_WPS.csv", sep="|")

//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (206,340) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [229]:
AHS_comb = pd.read_csv(data_path + "22_AHS_COMB.csv", sep="|")

//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [231]:
AHS_women = pd.read_csv(data_path + "22_AHS_WOMEN.csv", sep="|")

//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (118,147,178,185) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [91]:
AHS_mort_clean = remove_yellow_fields(AHS_mort, mort_field_list[0])

In [92]:
AHS_mort_clean

,house_no,house_hold_no,state,district,rural,stratum_code,psu_id,m_serial_no,deceased_sex,date_of_death,...,housestatus,householdstatus,isheadchanged,fidh,fidx,as,wt,schedule_id,year,v126
0,104,1,22,10,2,0,100660545,83,2,9,...,NaN,NaN,NaN,NaN,1.794122e+11,16.207033,26.639999,4,1,NaN
1,54,1,22,10,2,0,100659542,54,1,8,...,NaN,NaN,NaN,NaN,1.805322e+11,17.562895,26.639999,4,1,NaN
2,116,1,22,10,2,0,100659538,116,1,22,...,NaN,NaN,NaN,NaN,1.811522e+11,18.616802,26.639999,4,1,NaN
3,48,1,22,10,1,2,100660728,51,2,18,...,NaN,NaN,NaN,NaN,1.824722e+11,8.208295,22.129999,4,1,NaN
4,213,1,22,10,1,2,100660775,180,1,12,...,NaN,NaN,NaN,NaN,1.844122e+11,1.687450,22.129999,4,1,NaN
5,248,1,22,10,1,2,100660745,212,2,25,...,NaN,NaN,NaN,NaN,1.848422e+11,7.299030,22.129999,4,1,NaN
6,385,1,22,10,1,2,100660749,352,1,0,...,NaN,NaN,NaN,NaN,1.865922e+11,-0.567529,22.129999,4,1,NaN
7,115,2,22,10,1,1,100661091,98,2,15,...,NaN,NaN,NaN,NaN,1.884022e+11,-2.690575,33.220001,4,1,NaN
8,289,1,22,10,1,1,100661072,252,2,16,...,NaN,NaN,NaN,NaN,1.904022e+11,-1.357093,33.220001,4,1,NaN
9,455,1,22,10,1,1,100661073,377,2,0,...,NaN,NaN,NaN,NaN,1.921522e+11,-2.538779,33.220001,4,1,NaN


In [93]:
AHS_mort_clean_sorted = sort_dataset_state_dist_house(AHS_mort_clean)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [94]:
AHS_mort_clean_sorted

,house_no,house_hold_no,state,district,rural,stratum_code,psu_id,m_serial_no,deceased_sex,date_of_death,...,housestatus,householdstatus,isheadchanged,fidh,fidx,as,wt,schedule_id,year,v126
0,1,1,22,1,1,1,100635635,1,2,0,...,NaN,NaN,NaN,NaN,7.929220e+10,-4.786023,16.299999,4,1,NaN
1,1,1,22,1,1,1,100635396,1,1,8,...,NaN,NaN,NaN,NaN,8.298220e+10,-4.790645,16.299999,4,1,NaN
2,1,1,22,1,1,1,100635353,1,1,0,...,NaN,NaN,NaN,NaN,1.075322e+11,-2.521897,16.299999,4,1,NaN
3,1,1,22,1,1,1,100634959,2,1,13,...,1.0,1.0,2.0,2.201006e+14,NaN,NaN,16.299999,4,3,NaN
4,1,1,22,1,1,1,100634956,1,1,21,...,1.0,1.0,2.0,2.201006e+14,NaN,NaN,16.299999,4,3,NaN
5,1,1,22,1,1,1,100635466,1,2,0,...,1.0,1.0,2.0,2.201022e+14,NaN,NaN,16.299999,4,3,NaN
6,1,1,22,1,1,1,100635773,1,1,0,...,NaN,NaN,NaN,NaN,7.117220e+10,-1.845362,16.299999,4,1,NaN
7,1,1,22,1,1,1,100634957,1,2,0,...,NaN,NaN,NaN,NaN,6.692201e+09,-4.537548,16.299999,4,1,NaN
8,1,1,22,1,1,1,100634958,1,1,9,...,1.0,1.0,2.0,2.201006e+14,NaN,NaN,16.299999,4,2,NaN
9,1,2,22,1,1,2,100634897,391,2,1,...,1.0,1.0,2.0,2.201036e+14,NaN,NaN,8.860000,4,2,NaN


In [95]:
def district_wise_dataset(dataframe_state) :
    sorted_state = sort_dataset_state_dist_house(dataframe_state)
    unique_dist = sorted_state['district'].unique()
    out = list()
    
    for dist in unique_dist :
        dist_sort = (sorted_state[sorted_state['district'] == dist]).reset_index(drop=True)
        out.append([dist, dist_sort])
        
    return (out)

In [97]:
mort_dist_wise = district_wise_dataset(AHS_mort_clean_sorted)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [152]:
def one_hot_df(data_frame, one_hot_colnames) :
    colnames = list(data_frame)
    hot_col = list()
    
    for hot in one_hot_colnames :
        if hot in colnames :
            hot_col.append(hot)
            
    if 'district' in hot_col :
        hot_col.remove('district')
    if 'state' in hot_col :
        hot_col.remove('state')
            
    df = pd.get_dummies(data_frame, columns=hot_col)
    return (df)

In [101]:
mort___ = one_hot_df(mort_dist_wise[0][1], mort_field_list[4])

In [102]:
def house_no_wise_dataset(dist_data_frame) :
    unique_house = dist_data_frame['house_no'].unique()
    out = list()

    for house in unique_house :
        house_sort = (dist_data_frame[dist_data_frame['house_no'] == house]).reset_index(drop=True)
        out.append([house, house_sort])
    
    return (out)

In [103]:
mort_house_wise = house_no_wise_dataset(mort_dist_wise[0][1])

In [104]:
len(mort_house_wise)

342

In [105]:
mort_house_wise[0][1]

,house_no,house_hold_no,state,district,rural,stratum_code,psu_id,m_serial_no,deceased_sex,date_of_death,...,housestatus,householdstatus,isheadchanged,fidh,fidx,as,wt,schedule_id,year,v126
0,1,1,22,1,1,1,100635635,1,2,0,...,NaN,NaN,NaN,NaN,7.929220e+10,-4.786023,16.299999,4,1,NaN
1,1,1,22,1,1,1,100635396,1,1,8,...,NaN,NaN,NaN,NaN,8.298220e+10,-4.790645,16.299999,4,1,NaN
2,1,1,22,1,1,1,100635353,1,1,0,...,NaN,NaN,NaN,NaN,1.075322e+11,-2.521897,16.299999,4,1,NaN
3,1,1,22,1,1,1,100634959,2,1,13,...,1.0,1.0,2.0,2.201006e+14,NaN,NaN,16.299999,4,3,NaN
4,1,1,22,1,1,1,100634956,1,1,21,...,1.0,1.0,2.0,2.201006e+14,NaN,NaN,16.299999,4,3,NaN
5,1,1,22,1,1,1,100635466,1,2,0,...,1.0,1.0,2.0,2.201022e+14,NaN,NaN,16.299999,4,3,NaN
6,1,1,22,1,1,1,100635773,1,1,0,...,NaN,NaN,NaN,NaN,7.117220e+10,-1.845362,16.299999,4,1,NaN
7,1,1,22,1,1,1,100634957,1,2,0,...,NaN,NaN,NaN,NaN,6.692201e+09,-4.537548,16.299999,4,1,NaN
8,1,1,22,1,1,1,100634958,1,1,9,...,1.0,1.0,2.0,2.201006e+14,NaN,NaN,16.299999,4,2,NaN
9,1,2,22,1,1,2,100634897,391,2,1,...,1.0,1.0,2.0,2.201036e+14,NaN,NaN,8.860000,4,2,NaN


In [160]:
def recompile_district_dataset(house_level_data_list) :
    df_list = list()
    
    for i in range(len(house_level_data_list)) :
        house_df = house_level_data_list[i][1]
        df_list.append(house_df)
        
    dist_data = pd.concat(df_list)
    dist_data = dist_data.reset_index(drop=True)
    return(dist_data)

In [147]:
def recompile_state_dataset(dist_level_data_list) :
    df_list = list()
    
    for i in range(len(dist_level_data_list)) :
        dist_df = dist_level_data_list[i]
        df_list.append(dist_df)
        
    state_data = pd.concat(df_list)
    return(state_data)

In [107]:
dist__1 = recompile_district_dataset(house_no_wise_dataset(mort_dist_wise[0][1]))

In [108]:
dist__1

,house_no,house_hold_no,state,district,rural,stratum_code,psu_id,m_serial_no,deceased_sex,date_of_death,...,housestatus,householdstatus,isheadchanged,fidh,fidx,as,wt,schedule_id,year,v126
0,1,1,22,1,1,1,100635635,1,2,0,...,NaN,NaN,NaN,NaN,7.929220e+10,-4.786023,16.299999,4,1,NaN
1,1,1,22,1,1,1,100635396,1,1,8,...,NaN,NaN,NaN,NaN,8.298220e+10,-4.790645,16.299999,4,1,NaN
2,1,1,22,1,1,1,100635353,1,1,0,...,NaN,NaN,NaN,NaN,1.075322e+11,-2.521897,16.299999,4,1,NaN
3,1,1,22,1,1,1,100634959,2,1,13,...,1.0,1.0,2.0,2.201006e+14,NaN,NaN,16.299999,4,3,NaN
4,1,1,22,1,1,1,100634956,1,1,21,...,1.0,1.0,2.0,2.201006e+14,NaN,NaN,16.299999,4,3,NaN
5,1,1,22,1,1,1,100635466,1,2,0,...,1.0,1.0,2.0,2.201022e+14,NaN,NaN,16.299999,4,3,NaN
6,1,1,22,1,1,1,100635773,1,1,0,...,NaN,NaN,NaN,NaN,7.117220e+10,-1.845362,16.299999,4,1,NaN
7,1,1,22,1,1,1,100634957,1,2,0,...,NaN,NaN,NaN,NaN,6.692201e+09,-4.537548,16.299999,4,1,NaN
8,1,1,22,1,1,1,100634958,1,1,9,...,1.0,1.0,2.0,2.201006e+14,NaN,NaN,16.299999,4,2,NaN
9,1,2,22,1,1,2,100634897,391,2,1,...,1.0,1.0,2.0,2.201036e+14,NaN,NaN,8.860000,4,2,NaN


In [109]:
mort_dist_wise[0][1]

,house_no,house_hold_no,state,district,rural,stratum_code,psu_id,m_serial_no,deceased_sex,date_of_death,...,housestatus,householdstatus,isheadchanged,fidh,fidx,as,wt,schedule_id,year,v126
0,1,1,22,1,1,1,100635635,1,2,0,...,NaN,NaN,NaN,NaN,7.929220e+10,-4.786023,16.299999,4,1,NaN
1,1,1,22,1,1,1,100635396,1,1,8,...,NaN,NaN,NaN,NaN,8.298220e+10,-4.790645,16.299999,4,1,NaN
2,1,1,22,1,1,1,100635353,1,1,0,...,NaN,NaN,NaN,NaN,1.075322e+11,-2.521897,16.299999,4,1,NaN
3,1,1,22,1,1,1,100634959,2,1,13,...,1.0,1.0,2.0,2.201006e+14,NaN,NaN,16.299999,4,3,NaN
4,1,1,22,1,1,1,100634956,1,1,21,...,1.0,1.0,2.0,2.201006e+14,NaN,NaN,16.299999,4,3,NaN
5,1,1,22,1,1,1,100635466,1,2,0,...,1.0,1.0,2.0,2.201022e+14,NaN,NaN,16.299999,4,3,NaN
6,1,1,22,1,1,1,100635773,1,1,0,...,NaN,NaN,NaN,NaN,7.117220e+10,-1.845362,16.299999,4,1,NaN
7,1,1,22,1,1,1,100634957,1,2,0,...,NaN,NaN,NaN,NaN,6.692201e+09,-4.537548,16.299999,4,1,NaN
8,1,1,22,1,1,1,100634958,1,1,9,...,1.0,1.0,2.0,2.201006e+14,NaN,NaN,16.299999,4,2,NaN
9,1,2,22,1,1,2,100634897,391,2,1,...,1.0,1.0,2.0,2.201036e+14,NaN,NaN,8.860000,4,2,NaN


In [241]:
def make_dataset_district_wise(all_datasets_list, all_field_list, state_code) :  
    
    dataset_clean_sorted_list = list()
    
    for i in range(len(all_datasets_list)) :
        dataset = all_datasets_list[i]
        field = all_field_list[i]
        
        dataset_clean = remove_yellow_fields(dataset, field[0])
        dataset_clean_sorted = sort_dataset_state_dist_house(dataset_clean)
        
        dataset_clean_sorted_list.append(dataset_clean_sorted)
        
#     file_handler = open(str(state_code) + "_clean_sorted_data_list", "wb")
#     pickle.dump(dataset_clean_sorted_list, file_handler)
#     file_handler.close()
    

    dataset_wise_district_list = list()
    
    for i in range(len(dataset_clean_sorted_list)) :
        unique_dist = dataset_clean_sorted_list[i]['district'].unique()
        field = all_field_list[i]
        dataset = dataset_clean_sorted_list[i]
        
        dist_final_list = list()
        dist_dataset_list = district_wise_dataset(dataset)
        
        j = 0
        for dist in unique_dist :
            dist_j = dist_dataset_list[j][1]
            dist_j_hot = one_hot_df(dist_j, field[4])
            dist_j_hot_house = house_no_wise_dataset(dist_j_hot)
            # Replace empty values by <NaN>
            dist_j_final = recompile_district_dataset(dist_j_hot_house)
            dist_j_final = dist_j_final.replace(r'\s+', np.nan, regex=True)

            csv_name = str(str(i) + "_" + str(state_code) + "_" + str(dist) + ".csv")

            
            file_path = str(data_path + str(state_code) + "/")
            if not os.path.exists(file_path):
                os.makedirs(file_path)
                
            file_path = file_path + str(i) + "/"
            if not os.path.exists(file_path):
                os.makedirs(file_path)
            
            file_name = file_path + csv_name
            dist_j_final.to_csv(file_name)
            
            print(file_name, "written")
            
            dist_final_list.append(dist_j_final)
            j += 1
            
        dataset_wise_district_list.append(dist_final_list)
    
    return(dataset_wise_district_list)
    

In [232]:
csv_list = [AHS_mort, AHS_wps, AHS_women, AHS_comb]

In [234]:
field_list = [mort_field_list, wps_field_list, women_field_list, comb_field_list]

In [242]:
AHS_all_dist_wise = make_dataset_district_wise(csv_list, field_list, 22)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


('/Volumes/WD HDD/AHS_Data/22/0/0_22_1.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_2.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_3.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_4.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_5.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_6.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_7.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_8.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_9.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_10.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_11.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_12.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_13.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_14.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_15.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_16.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/1/1_22_1.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/1/1_22_2.cs

In [240]:
AHS_mort_dist_wise_hot = make_dataset_district_wise([AHS_mort.iloc[:10000]], [mort_field_list], 22)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


('/Volumes/WD HDD/AHS_Data/22/0/0_22_1.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_2.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_3.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_4.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_5.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_6.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_7.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_8.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_9.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_10.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_11.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_12.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_13.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_14.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_15.csv', 'written')
('/Volumes/WD HDD/AHS_Data/22/0/0_22_16.csv', 'written')


In [155]:
AHS_mort_dist_wise_hot[0][10].shape

(1024, 332)

In [161]:
state_data = recompile_state_dataset(AHS_mort_dist_wise_hot[0])

In [162]:
state_data.shape

(10000, 398)

In [163]:
state_data

,age,age_of_death_above_one_year,age_of_death_below_eleven_month,age_of_death_below_one_month,alcohol_1.0,alcohol_2.0,alcohol_3.0,alcohol_4.0,as,building_no,...,water_filteration_4.0,water_filteration_5.0,water_filteration_6.0,water_filteration_7.0,water_filteration_8.0,wt,year,year_of_birth,year_of_death,year_of_marriage
0,65.0,NaN,8.0,NaN,0,0,0,1,-4.786023,NaN,...,0,0.0,0.0,NaN,NaN,16.299999,1,1944.0,2009,1966.0
0,35.0,49.0,NaN,NaN,1,0,0,0,8.460409,1,...,0,0.0,0.0,NaN,NaN,16.299999,1,1974.0,2008,1990.0
1,72.0,65.0,NaN,NaN,0,1,0,0,8.023195,NaN,...,1,0.0,0.0,NaN,NaN,12.630000,1,1937.0,2007,1958.0
0,57.0,33.0,NaN,NaN,0,1,0,0,7.609506,NaN,...,0,0.0,0.0,NaN,NaN,16.299999,1,1952.0,2009,1982.0
1,27.0,14.0,NaN,NaN,0,0,0,1,-3.392867,NaN,...,0,0.0,0.0,NaN,NaN,16.299999,1,1982.0,2007,2002.0
0,57.0,47.0,NaN,NaN,0,0,0,1,NaN,NaN,...,0,0.0,0.0,NaN,NaN,8.860000,3,1954.0,2011,1979.0
1,48.0,20.0,NaN,NaN,0,0,0,1,NaN,NaN,...,1,0.0,0.0,NaN,NaN,12.630000,3,1963.0,2011,1981.0
2,27.0,41.0,NaN,NaN,1,0,0,0,-4.811838,NaN,...,0,0.0,0.0,NaN,NaN,8.860000,1,1982.0,2007,2007.0
0,48.0,55.0,NaN,NaN,0,0,0,1,9.369745,NaN,...,1,0.0,0.0,NaN,NaN,12.630000,1,1961.0,2008,1988.0
1,35.0,35.0,NaN,NaN,0,1,0,0,-5.126647,NaN,...,0,0.0,0.0,NaN,NaN,16.299999,1,1974.0,2008,1988.0


In [159]:
list(AHS_mort_dist_wise_hot[0][10])

['house_no',
 'house_hold_no',
 'state',
 'district',
 'psu_id',
 'm_serial_no',
 'date_of_death',
 'month_of_death',
 'year_of_death',
 'age_of_death_below_one_month',
 'age_of_death_below_eleven_month',
 'age_of_death_above_one_year',
 'serial_num_of_infant_mother',
 'order_of_birth',
 'is_death_associated_with_pregnan',
 'months_of_pregnancy',
 'time_between_onset_of_complicati',
 'nearest_medical_facility',
 'field38',
 'member_identity',
 'father_serial_no',
 'mother_serial_no',
 'date_of_birth',
 'month_of_birth',
 'year_of_birth',
 'age',
 'date_of_marriage',
 'month_of_marriage',
 'year_of_marriage',
 'building_no',
 'no_of_dwelling_rooms',
 'fid',
 'isdeadmigrated',
 'iscoveredbyhealthscheme',
 'healthscheme_1',
 'healthscheme_2',
 'housestatus',
 'householdstatus',
 'isheadchanged',
 'fidh',
 'fidx',
 'as',
 'wt',
 'schedule_id',
 'year',
 'v126',
 u'rural_1',
 u'rural_2',
 u'stratum_code_0',
 u'stratum_code_1',
 u'stratum_code_2',
 u'deceased_sex_1',
 u'deceased_sex_2',
 u't

In [113]:
asd = AHS_mort.iloc[:10000]

In [114]:
asd.shape

(10000, 122)

In [122]:
# pd.get_dummies(asd, columns=mort_field_list[4])
one_hot_df(asd, mort_field_list[4])

,id,m_id,client_m_id,hl_id,house_no,house_hold_no,psu_id,m_serial_no,date_of_death,month_of_death,...,land_possessed_2.0,land_possessed_3.0,land_possessed_4.0,land_possessed_5.0,land_possessed_6.0,residancial_status_1.0,residancial_status_2.0,residancial_status_3.0,residancial_status_4.0,residancial_status_6.0
0,396272,1165.0,236.0,17941.0,104,1,100660545,83,9,9,...,0,0,0,0,1,0,0,0,0,0
1,396280,1173.0,164.0,18053.0,54,1,100659542,54,8,7,...,0,0,0,0,1,0,0,0,0,0
2,396288,1181.0,172.0,18115.0,116,1,100659538,116,22,7,...,0,0,0,0,1,0,0,0,0,0
3,396296,1188.0,391.0,18247.0,48,1,100660728,51,18,8,...,0,0,0,0,0,0,0,0,0,0
4,396304,1196.0,399.0,18441.0,213,1,100660775,180,12,4,...,0,0,0,0,0,0,0,0,0,0
5,396312,1204.0,407.0,18484.0,248,1,100660745,212,25,4,...,0,0,1,0,0,0,0,0,0,0
6,396320,1212.0,415.0,18659.0,385,1,100660749,352,0,2,...,1,0,0,0,0,0,0,0,0,0
7,396328,1221.0,325.0,18840.0,115,2,100661091,98,15,7,...,1,0,0,0,0,0,0,0,0,0
8,396336,1229.0,333.0,19040.0,289,1,100661072,252,16,11,...,1,0,0,0,0,0,0,0,0,0
9,396344,1237.0,341.0,19215.0,455,1,100661073,377,0,10,...,1,0,0,0,0,0,0,0,0,0
